# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: GPT-2
* Evaluation approach: HuggingFace Trainer
* Fine-tuning dataset: sms_spam

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2"
)

tokenizer = AutoTokenizer.from_pretrained("gpt2")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from datasets import load_dataset

#ds = load_dataset("yelp_review_full")
ds = load_dataset("sms_spam")
ds = ds["train"].train_test_split(test_size=0.2)

ds["train"] = ds["train"]
ds["test"] = ds["test"]

In [3]:
splits = ["train", "test"]

tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = ds[split].map(
        lambda x: tokenizer(x["sms"], truncation=True, padding=True), batched=True
    )

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [4]:
model.config.pad_token_id = model.config.eos_token_id

import numpy as np
def compute_metrics(eval_prediction):
    predictions, labels = eval_prediction
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

training_args = TrainingArguments(
        output_dir=".data/sms_classifier",
        learning_rate=0.00002,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics
)

In [5]:
wolora_results = trainer.evaluate()
print(wolora_results)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 6.9648237228393555, 'eval_accuracy': 0.14439461883408072, 'eval_runtime': 15.5461, 'eval_samples_per_second': 71.722, 'eval_steps_per_second': 17.947}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [6]:
from peft import LoraConfig, get_peft_model, TaskType
lora_config = LoraConfig(task_type=TaskType.SEQ_CLS)
lora_model = get_peft_model(model, lora_config)

lora_model.print_trainable_parameters()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 297,984 || all params: 124,737,792 || trainable%: 0.23888830740245906


In [7]:
training_args.output_dir =".data/sms_classifier_lora"

train_lora = tokenized_dataset['train'].rename_column('label', 'labels')
test_lora = tokenized_dataset['test'].rename_column('label', 'labels')
    
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_lora,
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.332700,0.309726,0.864574


Checkpoint destination directory .data/sms_classifier_lora/checkpoint-1115 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1115, training_loss=1.1623824970604593, metrics={'train_runtime': 183.1528, 'train_samples_per_second': 24.346, 'train_steps_per_second': 6.088, 'total_flos': 540990279972864.0, 'train_loss': 1.1623824970604593, 'epoch': 1.0})

In [8]:
lora_model.save_pretrained("gpt-lora")


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [9]:
from peft import AutoPeftModelForSequenceClassification
loaded_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
loaded_model = model.to("cuda")
trainer.model = loaded_model
wlora_results = trainer.evaluate()

In [11]:
print("Accuracy before adding and training the LoRA adaptor: " + str(wolora_results['eval_accuracy']))
print("Accuracy after adding and training the LoRA adaptor: " + str(wlora_results['eval_accuracy']))

Accuracy before adding and training the LoRA adaptor: 0.14439461883408072
Accuracy after adding and training the LoRA adaptor: 0.8645739910313901
